In [2]:
# notebook for settings generation

In [3]:
import urdubiometer as ubm
from graphtransliterator import GraphTransliterator
urduGT = GraphTransliterator.from_yaml_file("../../transliterate/urdu.yml")
scholarlyGT = GraphTransliterator.from_yaml_file("../../transliterate/transliteration.yml")
hindiGT = GraphTransliterator.from_yaml_file('../../transliterate/devanagari.yaml')

/Users/seanpue/MyWorkspace/urdubiometer/urdubiometer/graphparser/validate.py:56: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  onmatch_regex=ONMATCH_RE.pattern)


In [4]:
lines = '''bol kih lab aazaad hai;n tere
bol zabaa;n ab tak terii hai
teraa sutvaa;n jism hai teraa
bol kih jaa;n ab tak terii hai
dekh kih aahangar kii dukaa;n me;n
tund hai;n shole sur;x hai aahan
khulne lage quflo;n ke dahaane
phailaa har ik zanjiir kaa daaman
bol yah tho;raa vaqt bahut hai
jism-o-zabaa;n kii maut se pahle
bol kih sach zindah hai ab tak
bol jo kuchh kahnaa hai kah le'''.split('\n')

lines_en = '''Speak, for your lips are free
Speak, for your tongue is still yours
Your long-suffering body is yours
Speak, for your life is still your own
Speak, for in the blacksmith's shop
The flames are fierce, the iron red
The mouths of locks have begun to open
The skirt of every chain is outspread
Speak, this little time is enough
Before the death of the body and tongue
Speak, for truth is still alive
Speak, say what must be said
'''.split("\n")

lines_ur = [urduGT.transliterate(line) for line in lines]
lines_trans = [scholarlyGT.transliterate(line) for line in lines]
lines_hi = [hindiGT.transliterate(line) for line in lines]

In [5]:
def vocalize(line):
    def show_arab(token):
        replacements = [ ('a', '^a'),
                         ('i', '^i'),
                         ('u', '^u'),
                         ('ai', '^ai'),
                         ('au', '^au'),
                         ('uu', '^uu')]
        for a, b in replacements:
            if token == a:
                token = b
        return token
    return ''.join([show_arab(token) for token in urduGT.tokenize(line)])


In [6]:
scanner = ubm.DefaultScanner(meters_list=[{'id':0, 'name': "Faiz", "regex_pattern": "=(=|--)===(=|--)=="}])


/Users/seanpue/MyWorkspace/urdubiometer/urdubiometer/graphparser/graphparser.py:541: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(yaml_str)
/Users/seanpue/MyWorkspace/urdubiometer/urdubiometer/graphparser/validate.py:105: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)
/Users/seanpue/MyWorkspace/urdubiometer/urdubiometer/scanner/default.py:42: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f.read())
/Users/seanpue/MyWorkspace/urdubiometer/urdubiometer/scanner/validate.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.

In [7]:
scans = [scanner.scan(line, graph_details=True)[0] for line in lines]

In [9]:
labels_of = {} # "1" -> verse 1

for scan_id, scan in enumerate(scans):
    labels = {'orig': [], 'en': [], 'ur': [], 'hi': []}
    for match in scan.matches:
        orig_tokens = ''.join(match.orig_tokens)
        labels['orig'].append(orig_tokens)
        labels['en'].append(scholarlyGT.transliterate(orig_tokens).replace(" ", "·"))
        labels['ur'].append(urduGT.transliterate(orig_tokens)#.replace(" ", "·"))
        labels['hi'].append(hindiGT.transliterate(orig_tokens).replace(" ", "·")   )
    labels_of[scan_id+1] = labels

SyntaxError: invalid syntax (<ipython-input-9-6263a0c83578>, line 10)

In [20]:
tokens = set()
for scan in scans:
    for match in scan.matches:
        for _ in match.orig_tokens:
            tokens.add(_)
token_trans = {}
vowels_to_mark = set(['a', 'i', 'u', 'ii', 'au', 'ai'])
for token in tokens:
    token_trans[token] = {}
    
    token_trans[token]['en'] = scholarlyGT.transliterate(token)
    token_trans[token]['hi'] = hindiGT.transliterate(token)
    if token in vowels_to_mark:
        token_ur = '^' + token
    else:
        token_ur = token
    token_trans[token]['ur'] = urduGT.transliterate(token_ur)
    

In [21]:
settings = {
    'lines': {
        '_raw': lines,
        'ur': lines_ur,
        'en': lines_en,
        'hi': lines_hi
    },
    'scans': {str(i+1):scan for i, scan in enumerate(scans)},
    'labels_of': labels_of,
    'token_trans': token_trans
}

import json
with open("../settings.js", "w") as f:
    f.write("settings="+json.dumps(settings))
